# python & ome.zarr

[NGFF](https://ngff.openmicroscopy.org/latest/) is a community driven effort to develop standardized data formats for microscopy that are performant and cloud ready. The primary format is `ome.zarr`, which is based on the popular [zarr](https://github.com/zarr-developers/zarr-python) n-dimensonial array data format.
The development effort is ongoing, but the specification for n-dimensional image data is already present (version 0.3 is under development at the time of writing this notebook) and ready for use.
For more information check out [this preprint](https://www.biorxiv.org/content/10.1101/2021.03.31.437929v4).

Two python packages exist to make life easier for python developers working with the ome.zarr data format:
- https://github.com/ome/ome-zarr-py for reading, writing and other functionality
- https://github.com/ome/napari-ome-zarr for displaying ome.zarr files in napari.

In this notebook, we will use them to create data in the ome.zarr format, read it back into memory, display it with napari and explore additional functionality.

NOTE: both this tutorial and the ome-zarr library are WIP and some of the features used in the tutorial may only work on the current main branch and not in the pip package yet.

## Installation

### Via pip

You can install both packages with pip:
```
$ pip install ome-zarr
$ pip install napari-ome-zarr
```

### Setting up conda envs

You can find two conda environment files in this gist: `environment.yaml` for a standard environment and `devel_env.yaml` for a development environment.

Check out [the miniconda documentation](https://docs.conda.io/en/latest/miniconda.html) if you need to install conda first.

The default environment can be set up via
```
$ conda env create -f environment.yaml
```
and then activated via
```
$ conda activate ome-zarr-py
```

To set up the development environment, first run
```
$ conda env create -f devel_env.yaml
```
then activate it via
```
$ conda activate ome-zarr-py-dev
```
and clone the two repositories https://github.com/ome/ome-zarr-py, https://github.com/ome/napari-ome-zarr. Install them by running 
```
$ pip install --no-deps -e .
```
inside both top-level folders. At the time of writing support for spec v0.3 is still under development, but can be used experimentally by using these two PRs instead of the main branches: https://github.com/ome/ome-zarr-py/pull/89, https://github.com/ome/napari-ome-zarr/pull/8.

## Writing ome.zarr data

Write an example image, represented in memory by a numpy array, to an ome.zarr file on disc.

In [ ]:
# the ome_zarr imports we require
from czitools.metadata import pylibczirw_metadata as czimd
from czitools.imagedata import pylibczirw_tools as czird
from czitools.utils import misc, napari_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
import napari
from pathlib import Path
import os
import ome_zarr.reader
import ome_zarr.scale
import ome_zarr.writer

# additional imports
import zarr
from skimage.data import astronaut  # example data

image = astronaut().transpose((2, 1, 0))  # transpose to channel first
# check the image shape, it should be cyx
print("Image shape:", image.shape)

In [ ]:
to_5d = True  # convert the data to normalized 5d axes?

ngff_version = ome_zarr.format.CurrentFormat().version
print("Using ngff format version", ngff_version)

# versions 0.1 and 0.2 only support normalized 5d
if int(ngff_version.split('.')[1]) <= 2:
    to_5d = True

if to_5d:
    axes = ("t", "c", "z", "y", "x")
    print("Convert data to tczyx")
    image = image[None, :, None]  # insert singleton t and z axis
    print("New shape:", image.shape)
else:
    axes = ("c", "y", "x")
    print("Keep data as cyx")

In [ ]:
# create multiscale image pyramid (mip) using the scaler class
scaler = ome_zarr.scale.Scaler()
# TODO support <5d
# TODO how do we control downscaling levels and options?
mip = scaler.local_mean(image)

In [ ]:
file_path = r"c:/temp/ome_zarr/my-first.ome.zarr"  # where to save the ome.zarr file
# create a zarr handle in write mode
# WARNING: this will delete everything in 'file_path'.
# if you don't want this, use mode="a" instead
loc = ome_zarr.io.parse_url(file_path, mode="w")  # FIXME 'w' does not truncate!

# create a zarr root level group at the file path
group = zarr.group(loc.store)

# write the actual data
ome_zarr.writer.write_multiscale(mip, group)

In [ ]:
# convince yourself that the data is there
import os
print(os.listdir(file_path))

## Reading ome.zarr data

Read back the example data from the ome.zarr file into memory.

In [ ]:
loc = ome_zarr.io.parse_url(file_path, mode="r")  # open the file in read mode
# this will return a reader object, which enables access to the indvidual resolution levels 
zarr_reader = ome_zarr.reader.Reader(loc).zarr

In [ ]:
# TODO is there a way to list the available resolution arrays?
# the 'load' functionality returns the specified resolution data as a dask array
res0 = zarr_reader.load("0")

In [ ]:
# the dask array can be used for lazy computation, or converted to numpy via .compute()
# for more information on dask arrays check out https://docs.dask.org/en/latest/array.html
full_image_npy = res0.compute()
print(full_image_npy.shape)

In [ ]:
# data slices can be used to select parts of the image.
# these will also be returned as dask arrays
sub_image = res0[:, 0, :, :256, :256]
sub_image_npy = sub_image.compute()
print(sub_image_npy.shape)

## Using napari with ome.zarr

Use the napari plugin installed with `napari-ome-zarr` to open ome.zarr files directly.

In [ ]:
# this is how we can open the file we just wrote in napari
import napari
viewer = napari.Viewer()
viewer.open(file_path)

## More functionality

The `ome_zarr` library provides additional functionality, like command line tools for inspecting and downloading ome.zarr data from s3.

In [ ]:
# inspect ome.zarr data on s3
!ome_zarr info 'https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001240.zarr/'

In [ ]:
# download the same data from s3
!ome_zarr download 'https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001240.zarr/'

In [ ]:
# we can also open the ome.zarr file directly in napari via the command line
!napari '6001240.zarr'

In [ ]:
# or we can pass the s3 address to open it on demand
!napari 'https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001240.zarr/'